# Quantum Neural Network

QNNはいくつかの種類に分けることが出来て問題に応じて向き不向きがある。
- NeuralNetwork
- OpflowQNN
- TwoLayerQNN
- CircuitQNN

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.opflow import Z, I, StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient
from qiskit.utils import QuantumInstance
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B

In [2]:
import qiskit.tools.jupyter
%qiskit_version_table

In [3]:
expval = AerPauliExpectation()
gradient = Gradient()
qi_sv = QuantumInstance(Aer.get_backend('statevector_simulator'))
qi_qasm = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=10)

## OpflowQNN

In [5]:
from qiskit_machine_learning.neural_networks import OpflowQNN

In [11]:
params1 = [Parameter('input1'), Parameter('weight1')]
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.ry(params1[0], 0)
qc1.rx(params1[1], 0)

param_dict = dict(zip(params1, [input1[0], weight1[0]]))
qc1.assign_parameters(param_dict, inplace=True)

qc_sfn1 = StateFn(qc1)

H1 = StateFn(PauliSumOp.from_list([('Z', 1.0), ('X', 1.0)]))

op1 = ~H1 @ qc_sfn1

In [8]:
qnn1 = OpflowQNN(op1, [params1[0]], [params1[1]], expval, gradient, qi_sv)

input1 = np.random.rand(qnn1.num_inputs)
weight1 = np.random.rand(qnn1.num_weights)

In [15]:
qnn1.forward(input1, weight1)

array([[-0.14251869]])

In [16]:
qnn1.forward([input1, input1], weight1)

array([[-0.14251869],
       [-0.14251869]])

In [14]:
# forwardは期待値を計算しているだけ
from qiskit.opflow.converters import CircuitSampler
expectation = expval.convert(op1)
sampler = CircuitSampler(qi_qasm).convert(expectation) 
sampler.eval().real

-0.14251869430838415

In [17]:
# output: input gradiend, weight gradient
qnn1.backward(input1, weight1)

(array([[[-1.29668149]]]), array([[[0.45449843]]]))

In [18]:
qnn1.backward([input1, input1], weight1)

(array([[[-1.29668149]],
 
        [[-1.29668149]]]),
 array([[[0.45449843]],
 
        [[0.45449843]]]))

## Classifier

In [ ]:
from qiskit_machine_learning.neural_networks import OpflowQNN, TwoLayerQNN, CircuitQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier, VQC

In [ ]:
quantum_instance = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=1024)

データを準備する。20個のサンプルについて、$y=x$を境界として$y>x$の範囲に存在するか、$y<x$の範囲に存在するかを分類する。

In [ ]:
num_inputs = 2
num_samples = 20
X = 2*np.random.rand(num_samples, num_inputs) - 1 # num_sample x num_inpuutsの配列
y01 = 1*(np.sum(X, axis=1) >= 0)  # in { 0,  1}
y = 2*y01-1                       # in {-1, +1}
y_one_hot = np.zeros((num_samples, 2))
for i in range(num_samples):
    y_one_hot[i, y01[i]] = 1

for x, y_target in zip(X, y):
    if y_target == 1:
        plt.plot(x[0], x[1], 'bo')
    else:
        plt.plot(x[0], x[1], 'go')
plt.plot([-1, 1], [1, -1], '--', color='black')
plt.show()

`OpflowQNN`を使った分類器を構築する。ただし`OpflowQNN`の出力はだと[-1, +1]の範囲内の一次元配列であるため、二値分類のみでしか利用できない。<br>
今回はOpflowQNNを拡張したTwoLayerQNNを利用する

In [ ]:
opflow_qnn = TwoLayerQNN(num_inputs, quantum_instance=quantum_instance)
opflow_classifier = NeuralNetworkClassifier(opflow_qnn, optimizer=COBYLA())

学習させて誤差を評価する

In [ ]:
opflow_classifier.fit(X, y)
opflow_classifier.score(X, y)

結果を可視化する

In [ ]:
# evaluate data points
y_predict = opflow_classifier.predict(X)

# plot results
# red == wrongly classified
for x, y_target, y_p in zip(X, y, y_predict):
    if y_target == 1:
        plt.plot(x[0], x[1], 'bo')
    else:
        plt.plot(x[0], x[1], 'go')
    if y_target != y_p:
        plt.scatter(x[0], x[1], s=200, facecolors='none', edgecolors='r', linewidths=2)
plt.plot([-1, 1], [1, -1], '--', color='black')
plt.show()